import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

In [1]:
from sqlalchemy import create_engine
import pandas as pd


In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

con = engine.connect()
rs = con.execute("SELECT * FROM Facilities")
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()


df=pd.read_sql_query('SELECT * FROM Members',engine)
df

<font color=red>or you can use below short entry using PANDAS POWER

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [13]:
df = pd.read_sql_query("""
                        SELECT f.name AS facility_name, 
                        SUM(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost ELSE b.slots * f.membercost END) AS revenue 
                      FROM Facilities AS f 
                      INNER JOIN Bookings AS b 
                        ON f.facid = b.facid 
                      GROUP BY f.name 
                      HAVING revenue < 1000 
                      ORDER BY revenue;"""
                       
                       ,engine)
                    

In [14]:
df

,facility_name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [5]:
df=pd.read_sql_query('SELECT surname, firstname , recommendedby \
                      FROM Members\
                      ORDER BY surname,firstname;',engine)

In [6]:
df

,surname,firstname,recommendedby
0,Bader,Florence,9
1,Baker,Anne,9
2,Baker,Timothy,13
3,Boothe,Tim,3
4,Butters,Gerald,1
5,Coplin,Joan,16
6,Crumpet,Erica,2
7,Dare,Nancy,4
8,Farrell,David,
9,Farrell,Jemima,


In [7]:
query = '''
SELECT DISTINCT (m1.surname || ', ' || m1.firstname) AS member_name, (m2.surname || ', ' || m2.firstname) AS reccomended_by
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY 1, 2;
'''

In [8]:
df = pd.read_sql_query(query, engine)

In [9]:
df

,member_name,reccomended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


/* Q12: Find the facilities with their usage by member, but not guests */

In [15]:
query = '''
SELECT f.name AS facility_name, m.firstname || ' ' || m.surname AS member_name, SUM(b.slots) AS usage_by_member
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY f.name, m.surname, m.firstname
ORDER BY usage_by_member DESC, facility_name, member_name;
'''

In [16]:
df = pd.read_sql_query(query, engine)

In [17]:
df

,facility_name,member_name,usage_by_member
0,Badminton Court,Darren Smith,432
1,Pool Table,Tim Rownam,282
2,Massage Room 1,Tim Rownam,176
3,Tennis Court 1,Gerald Butters,171
4,Tennis Court 2,Tim Boothe,168
...,...,...,...
197,Table Tennis,Anne Baker,2
198,Table Tennis,Gerald Butters,2
199,Table Tennis,John Hunt,2
200,Pool Table,Charles Owen,1


/* Q13: Find the facilities usage by month, but not guests */

# using SQLite  relational database management system,

In [18]:
with engine.connect() as con:
    rs=con.execute("""
                SELECT f.name AS facility_name,strftime('%Y-%m', b.starttime) AS year_month,SUM(b.slots) AS usage 
                 FROM Facilities AS f 
                 INNER JOIN Bookings AS b 
                  ON f.facid=b.facid
                 WHERE b.memid <> 0 
                 GROUP BY  facility_name,year_month 
                 Order By usage DESC;
                 """)
                  
                  
    d13=pd.DataFrame(rs.fetchall())
    d13.columns=rs.keys()

 

In [19]:
d13

,facility_name,year_month,usage
0,Badminton Court,2012-09,507
1,Pool Table,2012-09,443
2,Tennis Court 1,2012-09,417
3,Badminton Court,2012-08,414
4,Tennis Court 2,2012-09,414
5,Snooker Table,2012-09,404
6,Massage Room 1,2012-09,402
7,Table Tennis,2012-09,400
8,Tennis Court 2,2012-08,345
9,Tennis Court 1,2012-08,339
